# Devinons l'humeur d'une musique

L'objectif de ce projet est de pouvoir automatiquement déterminer si une musique est triste ou bien joyeuse. Pour cela nous allons utiliser la librairie [librosa](https://librosa.org/doc/latest/index.html). Cette dernière nous permettra de visualiser et d'obtenir des informations sur les musiques.
Nous avons téléchargé 10 musiques "tristes" et 10 musiques "joyeuses". Elles sont placées dans le dossier ./data/musics/ et commence par a pour les musiques joyeuses et b pour les musiques tristes.

In [1]:
%load_ext autoreload
%autoreload 2
data_dir = "./data/"
from utilities import *

In [2]:
import librosa # for working with audio in python
import librosa.display # for waveplots, spectograms, etc
import soundfile as sf # for accessing file information
import IPython.display as ipd # for playing files within python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## A quoi ça ressemble ?
Notre première étape va être de visualiser nos données. Peut-être que ça nous donnera des idées de features.

In [3]:
musics = ls("./data/musics")
for music in musics:
    info = sf.info("./data/musics/" + music)
    print(info)

./data/musics/a01.wav
samplerate: 44100 Hz
channels: 2
duration: 03:51.410 min
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]
./data/musics/a02.wav
samplerate: 44100 Hz
channels: 2
duration: 03:22.524 min
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]
./data/musics/a03.wav
samplerate: 44100 Hz
channels: 2
duration: 04:8.848 min
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]
./data/musics/a04.wav
samplerate: 44100 Hz
channels: 2
duration: 04:26.774 min
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]
./data/musics/a05.wav
samplerate: 44100 Hz
channels: 2
duration: 03:17.741 min
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]
./data/musics/a06.wav
samplerate: 44100 Hz
channels: 2
duration: 02:46.348 min
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]
./data/musics/a07.wav
samplerate: 44100 Hz
channels: 2
duration: 03:48.252 min
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit

In [15]:
y,sr = librosa.load('./data/musics/a01.wav', offset=2, duration=3)

df = pd.DataFrame(y, columns=['Amplitude'])

df.index = [(1/sr)*i for i in range(len(df.index))]

df

231 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:

musics = load_musics(data_dir + "musics/")

In [6]:
df = pd.DataFrame({
	'darkness' : musics.apply(mean_amplitude),
	'classe': musics.index.map(lambda name: 1 if name[0] == 'a' else -1),
})

ValueError: If using all scalar values, you must pass an index